In [1]:
using DelimitedFiles
using Distances
using FileIO
using LinearAlgebra
using Random
using Statistics
using DataFrames
using DataFramesMeta
using Plots
using PyCall
using CSV

In [2]:
m21 = pyimport("music21") 

PyObject <module 'music21' from '/home/alfredo/.local/lib/python3.10/site-packages/music21/__init__.py'>

In [3]:
#functions from personal packages.
git_path = "/home/alfredo/Git/Harmony_Evolution/src"
include(joinpath(git_path,"CEGFunctions.jl"))
include(joinpath(git_path,"Constr_series.jl"))
include(joinpath(git_path,"IM_Functions.jl"))
include(joinpath(git_path,"StochasticFunctions.jl"))

get_stochastic_kseq (generic function with 1 method)

In [4]:
xml_path = "/home/alfredo/MusicPenn/Beethoven_HAnalysis/XMLFiles" #xml files 
adata_path = "/home/alfredo/Git/Harmony_Evolution/Beethoven_Annotations/" #annotated data
out_path = "/home/alfredo/Git/Harmony_Evolution/Beethoven_Annotations/"
xml_list = readdir(xml_path)
ann_files = readdir(adata_path)
adata_list = ann_files[findall(x -> occursin(r".tsv", x), ann_files)];

In [5]:
opus_number = map(x-> join(split(x, "_")[1:2],"_"), xml_list)
u_opus = unique(opus_number);

In [6]:
for xml_i in 1:length(xml_list)
    local fh_kseq
    #loading csv from midi
    piece_xml = m21.converter.parse(joinpath(xml_path,xml_list[xml_i]))


    #loading hand-annotation data
    piece_ann = CSV.read(joinpath(adata_path,adata_list[xml_i]), DataFrame) 
    df_piece =  get_xml_df(piece_xml)


    separated_measures = groupby(df_piece, :Measure)


    kseq = Any[]
    n_bar = Int[]
    unc_seq = Float64[]
    for m in separated_measures
        out = get_distance_to_keys(get_center_effect(Matrix(m)))
        push!(n_bar, m[1,:Measure])
        push!(kseq, out[1,1])
        push!(unc_seq, round(get_key_ent(out),digits=4))
    end
    #fundamental key
    fun_key = get_rank_freq(kseq)[1,1] #Getting the most repeated key (expected to be the global key)
    #fun_key = piece_ann[1,:global_key]
    try
        fh_kseq = funhar_seq(kseq, fun_key) #mapping the key sequence to a functional harmony sequence, taking as reference the global key
    catch
        try
            new_funkey = key_translate[fun_key]
            fh_kseq = funhar_seq(kseq, new_funkey)
        catch
            fun_key = get_rank_freq(kseq)[1,1]
            fh_kseq = funhar_seq(kseq, fun_key)
        end
    end
    nmea = piece_ann.measure
    fh_out = []
    unc_out = []
    kseq_out = []
    for n_m in nmea
        loc = findfirst(x-> x==n_m, n_bar)
        if !isnothing(loc)
            push!(fh_out, fh_kseq[loc])
            push!(unc_out, unc_seq[loc])
            push!(kseq_out, kseq[loc])
        else
            push!(fh_out, "N/A")
            push!(unc_out, "N/A")
            push!(kseq_out, "N/A")
        end
    end

    g_key = [get_rank_freq(kseq)[1,1] for i in length(fh_out)]
    #construct the output dataframe
    df_out = DataFrame(
        :Measure => piece_ann[!,:measure],
        :Global_Key => piece_ann[!,:global_key],
        :Local_Key => piece_ann[!,:local_key],
        :Chord => piece_ann[!,:chord],
        :Relative_Numeral => piece_ann[!,:numeral],
        :CoE_KeyCall => fh_out,
        :CoE_Uncert => unc_out,
        :CoE_KeySequence => kseq_out,
        :CoE_GlobalKey => g_key
    )
    #exporting...
    xml_file = xml_list[xml_i]
    name_out = "CoEKeyCallNEW-$(join([split(xml_file,".")[1] "csv"],"."))"
    CSV.write(joinpath(out_path,name_out), df_out, header=true)
    println("File $(xml_file) DONE!")
    flush(stdout)
end

File op127_no12_mov1.musicxml DONE!


File op127_no12_mov2.musicxml DONE!


File op127_no12_mov3.musicxml DONE!


File op127_no12_mov4.musicxml DONE!


File op130_no13_mov1.musicxml DONE!


...

File op74_no10_mov4.musicxml DONE!


File op95_no11_mov1.musicxml DONE!


File op95_no11_mov2.musicxml DONE!


File op95_no11_mov3.musicxml DONE!


File op95_no11_mov4.musicxml DONE!


In [7]:
csv_files = readdir(out_path)
compare_list = csv_files[findall(x-> occursin(r"NEW-",x),csv_files)];

In [8]:

unc_wrong = []
fwrongs = []
mwrongs = []
good_guess = []
for f in 1:length(compare_list)
    bars = 0
    g_guess = 0
    df_ann = CSV.read(joinpath(out_path,compare_list[f]),DataFrame)
    
    g_key = get_rank_freq(df_ann[!,:Local_Key])[1,1]
    ann_meas = groupby(df_ann[df_ann.Local_Key .==g_key,:],:Measure)
    #ann_meas = groupby(df_ann, :Measure)
    
    tn_m = length(ann_meas)
    for i in 1:tn_m
        
        ann_keys = vcat(ann_meas[i][:,:Local_Key]..., ann_meas[i][:,:Relative_Numeral]..., ann_meas[i][:,:Chord]...)
        #ann_keys = vcat(ann_meas[i][:,:Relative_Numeral]...)
        coe_key = ann_meas[i][1,:CoE_KeyCall]
        if !isempty(findall(x-> x==coe_key, coalesce.(ann_keys,"N/A")))
            g_guess += 1
        else
            push!(unc_wrong, ann_meas[i][1,:CoE_Uncert])
            push!(fwrongs, f)
            push!(mwrongs, i)
        end
        bars +=1
    end
    push!(good_guess, g_guess / bars)
end

In [11]:
println("the median accuracy for all the movements is: ", round(median(good_guess)*100,digits=3) ,"%")

the median accuracy for all the movements is: 68.435%
